In [14]:
import pandas as pd
import os

In [15]:
cwd = os.getcwd()
print(cwd)

/home/guido/code/data-Zee/MMM-project-lewagon/mmmproject/api


In [16]:
parent_dir = '/home/guido/code/data-Zee/MMM-project-lewagon'
data_dir = 'raw_data'
data_file = 'df_clean.csv'

In [17]:
data_path = os.path.join(parent_dir, data_dir, data_file)
print(data_path)

/home/guido/code/data-Zee/MMM-project-lewagon/raw_data/df_clean.csv


In [18]:
df = pd.read_csv(data_path)

In [19]:
df = df.drop(columns='Unnamed: 0')
df['Day'] = pd.to_datetime(df['Day'])
df.set_index('Day', inplace=True)
df = df.rename(columns={"fb_costs": "facebook", "google_costs": "google", "tt_costs": "tiktok"})

simple_df = df.drop(columns = ['fb_impressions', 'fb_clicks', 'google_impressions', 'google_clicks', 'tt_impressions', 'tt_clicks'])

X_simple = simple_df.drop(columns = ['orders', 'total_sales']) # 2 channels: facebook and google
y_simple = simple_df['total_sales'] # for now lets just use total_sales and exclude orders

In [20]:
simple_df

,orders,total_sales,facebook,google,tiktok
Day,,,,,
2021-07-01,10,414.85,257.01,1.17,0.0
2021-07-02,6,229.95,250.30,0.84,0.0
2021-07-03,10,459.89,248.59,0.72,0.0
2021-07-04,17,508.36,258.05,0.07,0.0
2021-07-05,13,399.88,257.47,0.38,0.0
...,...,...,...,...,...
2023-08-20,294,13616.97,2148.53,466.56,0.0
2023-08-21,356,15348.42,2210.01,400.58,0.0
2023-08-22,150,6054.53,1335.84,289.46,0.0


In [21]:
X_simple

,facebook,google,tiktok
Day,,,
2021-07-01,257.01,1.17,0.0
2021-07-02,250.30,0.84,0.0
2021-07-03,248.59,0.72,0.0
2021-07-04,258.05,0.07,0.0
2021-07-05,257.47,0.38,0.0
...,...,...,...
2023-08-20,2148.53,466.56,0.0
2023-08-21,2210.01,400.58,0.0
2023-08-22,1335.84,289.46,0.0


In [22]:
y_simple

Day
2021-07-01      414.85
2021-07-02      229.95
2021-07-03      459.89
2021-07-04      508.36
2021-07-05      399.88
                ...   
2023-08-20    13616.97
2023-08-21    15348.42
2023-08-22     6054.53
2023-08-23     4810.67
2023-08-24     4889.20
Name: total_sales, Length: 785, dtype: float64

In [25]:
pip install mamimo

  Obtaining dependency information for mamimo from https://files.pythonhosted.org/packages/87/76/2bf2f17c069da09780bed67b7dcfeb664abe8fab572426c0fef1731d4e0d/mamimo-0.4.3-py3-none-any.whl.metadata
  Using cached mamimo-0.4.3-py3-none-any.whl.metadata (7.1 kB)
Using cached mamimo-0.4.3-py3-none-any.whl (16 kB)
Note: you may need to restart the kernel to use updated packages.


In [26]:
from mamimo.carryover import ExponentialCarryover
from mamimo.saturation import ExponentialSaturation
from mamimo.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [27]:
# initiate model
adstock = ColumnTransformer(
    [
     ('facebook_pipe', Pipeline([
            ('carryover', ExponentialCarryover()),
            ('saturation', ExponentialSaturation())
     ]), ['facebook']),
     ('google_pipe', Pipeline([
            ('carryover', ExponentialCarryover()),
            ('saturation', ExponentialSaturation())
     ]), ['google']),
     ('tiktok_pipe', Pipeline([
            ('carryover', ExponentialCarryover()),
            ('saturation', ExponentialSaturation())
     ]), ['tiktok'])
])

model = Pipeline([
    ('adstock', adstock),
    ('regression', LinearRegression(positive=True))
])

In [28]:
model

Pipeline(steps=[('adstock',
                 ColumnTransformer(transformers=[('facebook_pipe',
                                                  Pipeline(steps=[('carryover',
                                                                   ExponentialCarryover()),
                                                                  ('saturation',
                                                                   ExponentialSaturation())]),
                                                  ['facebook']),
                                                 ('google_pipe',
                                                  Pipeline(steps=[('carryover',
                                                                   ExponentialCarryover()),
                                                                  ('saturation',
                                                                   ExponentialSaturation())]),
                                                  ['google']),
                                                 ('tiktok_pipe',
                                                  Pipeline(steps=[('carryover',
                                                                   ExponentialCarryover()),
                                                                  ('saturation',
                                                                   ExponentialSaturation())]),
                                                  ['tiktok'])])),
                ('regression', LinearRegression(positive=True))])

In [29]:
model.fit(X_simple,y_simple)

Pipeline(steps=[('adstock',
                 ColumnTransformer(transformers=[('facebook_pipe',
                                                  Pipeline(steps=[('carryover',
                                                                   ExponentialCarryover()),
                                                                  ('saturation',
                                                                   ExponentialSaturation())]),
                                                  ['facebook']),
                                                 ('google_pipe',
                                                  Pipeline(steps=[('carryover',
                                                                   ExponentialCarryover()),
                                                                  ('saturation',
                                                                   ExponentialSaturation())]),
                                                  ['google']),
                                                 ('tiktok_pipe',
                                                  Pipeline(steps=[('carryover',
                                                                   ExponentialCarryover()),
                                                                  ('saturation',
                                                                   ExponentialSaturation())]),
                                                  ['tiktok'])])),
                ('regression', LinearRegression(positive=True))])

In [30]:
import pickle

In [31]:
model.predict(X_simple)

array([1435.18592247, 1092.44479702,  936.9766213 , -321.76066378,
        380.23178307, 2224.81604805, 1163.45644644, 1443.90870812,
        301.4016359 ,  581.43870961, -243.92620379, 1790.64835948,
       2131.26565037, 1989.33265716, 1615.30746997, 2178.06545593,
       1687.86166961, 2055.59567139, 1780.11972833, 1858.73686006,
       2310.17433199, 2289.04809797, 1399.40927047,  746.10751227,
       2249.89695895, 2037.01635793,  866.48621977, 2187.10897868,
       2202.3134249 , 1579.59583475,  581.43870961,  746.10751227,
        880.86766201, 1795.83423478, 2301.49227214, 2104.13108641,
       1469.55975189,  240.17488112, 1940.86928312, 1858.73686006,
       1208.48494101, 2190.79513498, 2151.68414425, 2089.07098761,
       1730.04599781, 1774.77591961, 2188.35000491, 2166.92433703,
       2295.7767949 , 1889.36896936, 1649.27742818, 1371.62182648,
       1735.83487669, 2310.28647827, 2296.25133187, 2254.08422817,
       1494.45269274, 1641.76940802,  615.70281521, 1629.09960

In [1]:
# with open('pipelines/pipeline.pkl', 'wb') as file:
#         pickle.dump(model, file)

In [32]:
!ls ../../pipelines

pipeline-2023-08-31.pkl


In [33]:
pathpipeline = '../../pipelines/pipeline-2023-08-31.pkl'

In [36]:
pipeline_loaded = pickle.load(open(pathpipeline, 'rb'))


In [43]:
pd.DataFrame(simple_df.iloc[0]).T

,orders,total_sales,facebook,google,tiktok
2021-07-01,10.0,414.85,257.01,1.17,0.0


In [52]:
inputdata = simple_df.iloc[0].T.values

In [64]:
pipeline_loaded.predict(query_sale_predict("2021-07-01", inputdata[0], inputdata[1], inputdata[2], inputdata[3], inputdata[4]))

array([1435.18592247])

In [62]:
def query_sale_predict(date: str, total_sales: float, orders: int, facebook: float, google: float, tiktok: float):
    """
    Returns predicted sales data from "df" according to cost input data
    """
    
    dict_feat = {
        "total_sales": total_sales,
        "orders": orders, "google": google, 
        "facebook": facebook, "tiktok": tiktok
    }
    X_pred = pd.DataFrame(dict_feat, index=[date])
    
    return X_pred

In [63]:
query_sale_predict("2021-07-01", inputdata[0], inputdata[1], inputdata[2], inputdata[3], inputdata[4])

,total_sales,orders,google,facebook,tiktok
2021-07-01,10.0,414.85,1.17,257.01,0.0


In [54]:
[data for data in inputdata]

[10.0, 414.85, 257.01, 1.17, 0.0]

In [57]:
",".join([data for data in inputdata])

TypeError: sequence item 0: expected str instance, numpy.float64 found